# Greedy Ensemble example

### Preparation


In this notebook, we use cifar10 to show you the performance of greedy ensemble operator.

In [57]:
import torch
import torchvision
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

tr_set = torchvision.datasets.CIFAR10(root = "./datasets/cifar10", train = True, download = True, transform = transform)
te_set = torchvision.datasets.CIFAR10(root = "./datasets/cifar10", train = False, download = True, transform = transform)

batch_size = 64

tr_data = torch.utils.data.DataLoader(tr_set, batch_size = batch_size, shuffle = True)
te_data = torch.utils.data.DataLoader(te_set, batch_size = batch_size, shuffle = False)

We use get_feature() to extract images and labels from original dataset.

In [18]:

def get_feature(data_set):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(data_set,batch_size=64)):

            all_features.append(images)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

### Training

Here we select two different models for training

In [3]:
import towhee
from towhee import ops
op = ops.image_embedding.timm(model_name ='resnet50', num_classes=10)

In [40]:
op2 = ops.image_embedding.timm(model_name ='resnet101', num_classes=10)

In [4]:
from towhee.trainer.training_config import TrainingConfig

training_config = TrainingConfig(
    batch_size=64,
    epoch_num=15,
    output_dir='cifar10_output1'
)

In [41]:
training_config2 = TrainingConfig(
    batch_size=64,
    epoch_num=15,
    output_dir='cifar10_output2'
)

In [5]:
op.train(training_config, train_dataset=tr_set,eval_dataset=te_set)

2022-06-13 10:43:20,921 - 139635262112320 - trainer.py-trainer:319 - WARNING: TrainingConfig(output_dir='cifar10_output1', overwrite_output_dir=True, eval_strategy='epoch', eval_steps=None, batch_size=64, val_batch_size=-1, seed=42, epoch_num=15, dataloader_pin_memory=True, dataloader_drop_last=True, dataloader_num_workers=0, lr=5e-05, metric='Accuracy', print_steps=None, load_best_model_at_end=False, early_stopping={'monitor': 'eval_epoch_metric', 'patience': 4, 'mode': 'max'}, model_checkpoint={'every_n_epoch': 1}, tensorboard={'log_dir': None, 'comment': ''}, loss='CrossEntropyLoss', optimizer='Adam', lr_scheduler_type='linear', warmup_ratio=0.0, warmup_steps=0, device_str=None, sync_bn=False, freeze_bn=False)
[epoch 1/15] loss=2.106, metric=0.286, eval_loss=2.057, eval_metric=0.426: 100%|██████████| 781/781 [00:52<00:00, 14.79step/s]
[epoch 2/15] loss=1.537, metric=0.511, eval_loss=1.504, eval_metric=0.558: 100%|██████████| 781/781 [00:55<00:00, 14.03step/s]
[epoch 3/15] loss=1.179

In [44]:
op2.train(training_config2, train_dataset=tr_set,eval_dataset=te_set)

2022-06-13 14:19:55,680 - 139635262112320 - trainer.py-trainer:319 - WARNING: TrainingConfig(output_dir='cifar10_output2', overwrite_output_dir=True, eval_strategy='epoch', eval_steps=None, batch_size=64, val_batch_size=-1, seed=42, epoch_num=15, dataloader_pin_memory=True, dataloader_drop_last=True, dataloader_num_workers=0, lr=5e-05, metric='Accuracy', print_steps=None, load_best_model_at_end=False, early_stopping={'monitor': 'eval_epoch_metric', 'patience': 4, 'mode': 'max'}, model_checkpoint={'every_n_epoch': 1}, tensorboard={'log_dir': None, 'comment': ''}, loss='CrossEntropyLoss', optimizer='Adam', lr_scheduler_type='linear', warmup_ratio=0.0, warmup_steps=0, device_str=None, sync_bn=False, freeze_bn=False)
[epoch 1/15] loss=2.266, metric=0.167, eval_loss=2.254, eval_metric=0.209: 100%|██████████| 781/781 [01:24<00:00,  9.19step/s]
[epoch 2/15] loss=2.078, metric=0.269, eval_loss=2.059, eval_metric=0.308: 100%|██████████| 781/781 [01:23<00:00,  9.37step/s]
[epoch 3/15] loss=1.864

### Evaluating

In [19]:
images,labels = get_feature(te_set)

100%|██████████| 157/157 [00:01<00:00, 98.56it/s]


In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
imagestensor = torch.from_numpy(images).to(device)

Predict propability needs to be saved for greedy ensemble.

In [34]:
# out = op.trainer.predict(te_set)
out1 = op._op.trainer.predict(imagestensor)

In [45]:
out2 = op2._op.trainer.predict(imagestensor)

In [97]:
def evaluate(pre,labels):
    ans=0
    for i in range(len(pre)):
        predict_num = torch.argmax(torch.softmax(pre[i], dim=-1)).item()
        if(predict_num==labels[i]):
            ans = ans+1
    return ans/len(pre)*100.0


Check the scores from the two models above.

In [98]:
evaluate(out1,labels)

79.31

In [99]:
evaluate(out2,labels)

66.93

### Greedy Ensemble inference

In this part, we use greedy ensemble operator to generate a new predict from previous models.

In [58]:
preds = []
preds.append(out1.cpu().detach().numpy())
preds.append(out2.cpu().detach().numpy())
preds = np.array(preds)

In [114]:
from towhee import Entity,DataCollection
dc = towhee.DataCollection([Entity(ens=preds,groud_truth = labels)]).unstream()
dc = dc.greedy_ensemble[('ens','groud_truth'),'greedy_ens'](size=4,topk=1)

In [115]:
new_pred = torch.from_numpy(dc[0].greedy_ens)
label = torch.from_numpy(dc[0].groud_truth)
score = evaluate(new_pred,label)
score

79.72

We can see approvement on score after greedy ensemble by 0.41.